# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686513


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:03<00:42,  1.51s/it]

Rendering models:  10%|█         | 3/30 [00:04<00:41,  1.53s/it]

Rendering models:  13%|█▎        | 4/30 [00:04<00:29,  1.13s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:40,  1.63s/it]

...

Rendering models:  83%|████████▎ | 25/30 [00:10<00:00,  6.38it/s]

Rendering models:  87%|████████▋ | 26/30 [00:11<00:00,  6.34it/s]

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  6.32it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  7.17it/s]

DevionJ                                0.001459
dmackieson                             0.001911
c_monkey                               0.005609
smithkurtis                            0.000526
Petrusperes                            0.002402
CThomas                                0.079917
Dynamatt                               0.010950
Camelron                               0.000333
Roquwat                                0.002112
not-logged-in-5a8bb7be433678ade709     0.001163
not-logged-in-c5ba8e66058678e0842b     0.007840
not-logged-in-f82a6d15a1dafa9bbfae     0.000466
crush202020                            0.000540
not-logged-in-dddbb97cfd84354416f5     0.008307
not-logged-in-f3082f2a86c2c9f95d30    43.928572
41Letters                              0.000373
arad98                                 0.000996
rhondacard                             0.019099
Lavadude                               0.018608
equidad1                               0.000825
ShirleyL                               0

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())